# SPELL CORRECTION

18210980078 张屹

In [12]:
import re
import nltk
import numpy as np
from copy import deepcopy

In [38]:
def edits1(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces = [L + c + R[1:] for L, R in splits if R for c in letters]
    inserts = [L + c + R for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

In [39]:
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [77]:
def edittype(word,error):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    word,error = word.lower(),error.lower()
    for i in range(len(word)):
        if word == error[1:]:
            return error[0]+'|<s>', 'ins'
        if word[1:] == error:
            return '|'+word[0], 'del'
        if i >= len(error):
            return word[i]+'|'+error[i-1]+word[i], 'del'
        elif word[i] != error[i]:
            print(word,error)
            if word in [error[:i]+k+error[i:] for k in letters]:
                return error[i-1]+'|'+error[i-1]+word[i], 'del'
            elif word in [error[:i]+k+error[i+1:] for k in letters]:
                return error[i]+'|'+word[i], 'sub'
            elif word == error[:i]+error[i+1:] or word == error[:-1]:
                return word[i-1]+error[i]+'|'+word[i-1], 'ins'
            elif word[i]+ word[i+1] == error[i+1]+error[i]:
                return word[i+1]+word[i]+'|'+word[i]+word[i+1], 'trans'
    if len(word)<len(error):
        return word[-1]+error[-1]+'|'+word[-1], 'ins'

In [78]:
edittype('sam','asm')

sam asm


('as|sa', 'trans')

In [1]:
def editdistance(s1, s2): 
    len1 = len(s1)
    len2 = len(s2)
     
    matrix = [[i+j for j in range(len2 + 1)] for i in range(len1 + 1)]
#     print(matrix)
    for row in range(len1):
        for col in range(len2):
            comp = [matrix[row+1][col]+1, matrix[row][col+1]+1]
             
            if s1[row] == s2[col]:
                comp.append(matrix[row][col])
            else:
                comp.append(matrix[row][col]+1)
             
            if row > 0 and col > 0:
                if s1[row] == s2[col-1] and s1[row-1] == s2[col]:
                    comp.append(matrix[row-1][col-1]+1)
                     
            matrix[row+1][col+1] = min(comp)
             
    return matrix[len1][len2],matrix

In [2]:
editdistance('intention','execution')

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11], [3, 4, 5, 6, 7, 8, 9, 10, 11, 12], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13], [5, 6, 7, 8, 9, 10, 11, 12, 13, 14], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15], [7, 8, 9, 10, 11, 12, 13, 14, 15, 16], [8, 9, 10, 11, 12, 13, 14, 15, 16, 17], [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]]


(5,
 [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [1, 1, 2, 3, 4, 5, 6, 6, 7, 8],
  [2, 2, 2, 3, 4, 5, 6, 7, 7, 7],
  [3, 3, 3, 3, 4, 5, 5, 6, 7, 8],
  [4, 3, 4, 3, 4, 5, 6, 6, 7, 8],
  [5, 4, 4, 4, 4, 5, 6, 7, 7, 7],
  [6, 5, 5, 5, 5, 5, 5, 6, 7, 8],
  [7, 6, 6, 6, 6, 6, 6, 5, 6, 7],
  [8, 7, 7, 7, 7, 7, 7, 6, 5, 6],
  [9, 8, 8, 8, 8, 8, 8, 7, 6, 5]])

In [44]:
def known(words):
    return set(word for word in words if word in VOCAB)

In [74]:
def p_type(candidate, word):
    wrong_type = edittype(word,candidate)
    if wrong_type[0] in EDIT_P.keys():
        return EDIT_P[wrong_type[0]]
    else:
        return -100000

In [46]:
def prob1(candidate, word, sentence):
    if candidate in UNIGRAM.keys():
        p_unigram = UNIGRAM[candidate]
    else:
        p_unigram = -100000
    return p_unigram + p_type(candidate, word)

In [47]:
def BIGRAM_p(s):
    if s in BIGRAM.keys():
        return BIGRAM[s]
    else:
        return -100000

In [48]:
def prob2(candidate, j, sentence):
    if j==0:
        return BIGRAM_p(candidate+' '+sentence[j+1].lower())
    elif j==len(sentence)-1:
        return BIGRAM_p((sentence[j-2].lower()+' '+candidate))
    else:
        return BIGRAM_p(candidate+' '+sentence[j+1].lower()) + BIGRAM_p(sentence[j-1].lower()+' '+candidate)

In [107]:
def non_word_correct(sentence):
    wrong = 0
    for j in range(len(sentence)):
        word = sentence[j]
        if bool(re.search(r"[\d.,/'-]", word)) or word.lower() in VOCAB:
            continue
        # edit distance = 1
        word_lower = word.lower()
        candidates = known(edits1(word_lower))
        # edit distance = 2
        if len(candidates)==0:
            candidates = known(edits2(word_lower))
        p_flag = -200000
        right = word
        for candidate in candidates:
            # unigram prob
#             p = prob1(candidate, word, sentence) #* prob2(error, candidate)
            p = prob2(candidate,j, sentence) # + p_type(candidate,word_lower) #channel model for edits1
            if p > p_flag:
                p_flag = p
                right = candidate
        if not word.islower():
            flag = 0
            for each in word:
                flag += int(each.isupper())
            if flag == 1:
                right = right[0].upper()+right[1:]
            else:
                right = right.upper()
        sentence[j] = right # to do supper letters
        wrong += 1
    return wrong

In [94]:
def real_word_correct(sentence):
    for j in range(len(sentence)):
        word = sentence[j]
        if bool(re.search(r"[\d.,/'-]", word)):
            continue
        # edit distance = 1
        word_lower = word.lower()
        candidates = known(edits1(word_lower))
        if len(candidates)==0:
            candidates = known(edits2(word_lower))
#         candidates = candidates|known(edits2(word_lower))
        candidates.add(word_lower)
        p_flag = -200000
        right = word_lower
        for candidate in candidates:
            # bigram prob
            p = prob2(candidate,j, sentence)
            if p > p_flag:
                p_flag = p
                right = candidate
#         print(candidates,right)
        if right == word_lower:
            continue
        if word != word_lower:
            flag = 0
            for each in word:
                flag += int(each.isupper())
            if flag == 1:
                right = right[0].upper()+right[1:]
            else:
                right = right.upper()
        sentence[j] = right # to do supper letters
#         print(j,sentence,word,right)
        if right.lower() != word_lower:
            return None

In [95]:
def word_correct(sentences_origin,wrong_words_num, VOCAB):
    sentences = deepcopy(sentences_origin)
    for i in range(len(sentences)):
        sentence = sentences[i]
        wrong = non_word_correct(sentence)
        if wrong < wrong_words_num[i]:
            real_word_correct(sentence)
    return sentences

In [96]:
with open('vocab.txt', 'r') as f:
    VOCAB = [re.sub('[0-9,.!“”‘’/\—\–\-\'\"]', '', line.strip()) for line in f.readlines()]
VOCAB = set([each.lower() for each in VOCAB if each!=''])

In [97]:
with open('testdata.txt','r') as f:
    sentences = f.read().split('\n')
wrong_words_num = [int(each.split('\t')[1]) for each in sentences]
sentences_origin = [nltk.word_tokenize(each.split('\t')[2]) for each in sentences]

In [98]:
with open('channel_model.txt',encoding='UTF-8') as f:
    EDIT_P = {}
    l = f.readlines()
#     print(l[0:3])
    for each in l:
        each = each.strip().split('\t')
#         print(each)
        EDIT_P[each[0]] = np.log(int(each[1]))

In [100]:
def ngram(file, n):
    with open(file) as f:
        p = f.read()
    p = [[re.sub('[,.!“”‘’\—\–\'\"]', '', word).lower() for word in sentence.split()] for sentence in nltk.sent_tokenize(p)]
    output = {}
    for l in p:
        for i in range(len(l)-n+1):
            temp = " ".join(l[i:i+n])
            if temp not in output:
                output[temp] = 0
            output[temp] += 1
    return output

In [101]:
file = 'ans.txt'
UNIGRAM = ngram(file,1)
SUM = sum(UNIGRAM.values())
for each in UNIGRAM:
    UNIGRAM[each] = np.log(UNIGRAM[each]/SUM)
BIGRAM = ngram(file,2)
SUM = sum(BIGRAM.values())
for each in BIGRAM:
    BIGRAM[each] = np.log(BIGRAM[each]/SUM)

In [110]:
sentences_correct = word_correct(sentences_origin, wrong_words_num, VOCAB)

In [111]:
file = 'result.txt'
with open(file,'w') as f:
    for i,each in enumerate(sentences_correct):
        sentence = ' '.join(sentences_correct[i])
        f.write(str(i+1)+'\t'+sentence+'\n')

In [20]:
# none word edits1 candidates unigram 40% 
# edits2 candidates 45.7%
# upper letter 85.8%
# real word edits1  bigram 89.2%
# none word bigram 93.1%
# to do: 
# p_type 
# UNIGRAM for a larger corpus
# count BIGRAM Prob
# real word error